In [1]:
import os

In [2]:
%pwd

'd:\\ML\\ML_END_TO_END_PROJECT\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\ML\\ML_END_TO_END_PROJECT'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)


        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            source_url= config.source_url,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config




        

In [8]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size


In [9]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers = request.urlretrieve(
                url= self.config.source_url,
                filename= self.config.local_data_file
            )

            logger.info(f"{filename} download! with following info: \n{headers}")
        
        else:
            logger.info(f"File already exist of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory & function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-08-28 14:39:40,514: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-28 14:39:40,516: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-28 14:39:40,527: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-28 14:39:40,533: INFO: common: created directory at: artifacts]
[2024-08-28 14:39:40,533: INFO: common: created directory at: artifacts/data_ingestion]


[2024-08-28 14:39:43,083: INFO: 3360249704: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 21984
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "85a38be3d4c70e3b6dc97fc47873623da88c9b850e1526aeb57ff236bdd2815c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: AC2B:2664F7:1C1C65:2371E3:66CEE561
Accept-Ranges: bytes
Date: Wed, 28 Aug 2024 09:09:31 GMT
Via: 1.1 varnish
X-Served-By: cache-del21720-DEL
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1724836171.782445,VS0,VE254
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 35c9de9c640cb30b85331935ee96c334db74166b
Expires: Wed, 28 Aug 2024 09:14:31 GMT
Source-Age: 0

]
